# How to use FIC in Jupyter

First note that the kernel for this notebook is Node.js, as indicated in the top right.

Since I have it stored in the main folder of the repository, I can import tools from this repo like so:

In [1]:
const {
    OM, LC, Statement, Environment,
    MatchingProblem, MatchingSolution, Turnstile
} = require( './classes/all.js' )

## Creating OpenMath and LC objects

The usual `OM.simple()` factory function builds OpenMath objects, which have been taught to represent themselves using a simple encoding.

In [2]:
OM.simple( 'f(x,y)' )

OMNode {
  tree: { t: 'a', c: [ [Object], [Object], [Object] ], p: null }
}

All the usual building blocks for LCs work as well, and LC objects have been taught to represent themselves using standard LC notation.

In [3]:
new Environment( LC.fromString( 'f(x)' ), LC.fromString( '{ :A B }' ) )

<ref *1> Environment {
  attributes: { declaration: 'none', formula: false },
  parentNode: null,
  childList: [
    Statement {
      attributes: [Object],
      parentNode: [Circular *1],
      childList: [Array],
      dirty: false,
      className: 'Statement',
      containsAMetavariable: [Function: containsAMetavariable]
    },
    Environment {
      attributes: [Object],
      parentNode: [Circular *1],
      childList: [Array],
      dirty: false,
      className: 'Environment',
      containsAMetavariable: [Function: containsAMetavariable],
      isAQuantifier: false
    }
  ],
  dirty: false,
  className: 'Environment',
  containsAMetavariable: [Function: containsAMetavariable]
}

## Asking matching questions

You can create a matching problem by providing a list of pattern-expression pairs.  Here we provide just one.  Note that the `MatchingProblem` object knows to represent itself as a table of its defining pairs.

In [4]:
setCapitalsMetavars = ( inThis ) => {
    if ( inThis.identifier && inThis.identifier == inThis.identifier.toUpperCase() )
        inThis.isAMetavariable = true
    inThis.children().map( setCapitalsMetavars )
}

pattern = LC.fromString( 'f(A,B)' )
setCapitalsMetavars( pattern )

expression = LC.fromString( 'f(one,two)' )

P = new MatchingProblem( [ pattern, expression ] )
P

Pattern,Expression
"f(A,B)","f(one,two)"


If we ask it for its solutions, the result is an array, which doesn't have any special printing properties, so it doesn't come out very nicely:

In [5]:
P.getSolutions()

[ MatchingSolution { _mapping: { A: [Statement], B: [Statement] } } ]

But we can ask for the first element of the array, and then the `MatchingSolution` object knows to pretty-print itself as a table of metavariable instantiations.

In [6]:
P.getSolutions()[0]

Metavariable,Instantiation
A,one
B,two


## Using FIC

Let's build a rule of logic and ask if it can be used to prove something.

To do so, we build an auxiliary routine that recursively converts any capitalized identifier to a metavariable, just for a simple convention we'll use here.

In [7]:
rule = LC.fromString( '{ :X :Y and(X,Y) }' )
setCapitalsMetavars( rule )
rule

<ref *1> Environment {
  attributes: { declaration: 'none', formula: false },
  parentNode: null,
  childList: [
    Statement {
      attributes: [Object],
      parentNode: [Circular *1],
      childList: [],
      dirty: false,
      className: 'Statement',
      containsAMetavariable: [Function: containsAMetavariable]
    },
    Statement {
      attributes: [Object],
      parentNode: [Circular *1],
      childList: [],
      dirty: false,
      className: 'Statement',
      containsAMetavariable: [Function: containsAMetavariable]
    },
    Statement {
      attributes: [Object],
      parentNode: [Circular *1],
      childList: [Array],
      dirty: false,
      className: 'Statement',
      containsAMetavariable: [Function: containsAMetavariable]
    }
  ],
  dirty: false,
  className: 'Environment',
  containsAMetavariable: [Function: containsAMetavariable],
  isAQuantifier: false
}

We can then pose the following problem and ask for all ways that FIC derivation with matching can solve it (if any).

$$a, ~ b, ~ \{~{:}X~{:}Y~and(X,Y)~\} \vdash and(b,a)$$

In [8]:
T = new Turnstile(
    [ LC.fromString( 'a' ), LC.fromString( 'b' ), rule ],
    LC.fromString( 'and(b,a)' ) )
T

Turnstile {
  premises: [
    Statement {
      attributes: [Object],
      parentNode: null,
      childList: [],
      dirty: false,
      className: 'Statement',
      containsAMetavariable: [Function: containsAMetavariable]
    },
    Statement {
      attributes: [Object],
      parentNode: null,
      childList: [],
      dirty: false,
      className: 'Statement',
      containsAMetavariable: [Function: containsAMetavariable]
    },
    Environment {
      attributes: [Object],
      parentNode: null,
      childList: [Array],
      dirty: false,
      className: 'Environment',
      containsAMetavariable: [Function: containsAMetavariable],
      isAQuantifier: false
    }
  ],
  conclusion: Statement {
    attributes: { identifier: 'and' },
    parentNode: null,
    childList: [ [Statement], [Statement] ],
    dirty: false,
    className: 'Statement',
    containsAMetavariable: [Function: containsAMetavariable]
  }
}

In [9]:
ms = T.allDerivationMatches( { withProofs: true } )
ms

[
  MatchingSolution {
    _mapping: { X: [Statement], Y: [Statement] },
    proof: Proof { turnstile: [Turnstile], rule: 'GL', subproofs: [Array] }
  }
]

Apparently there is one way, but again, it's reported as an array of one solution, so it didn't pretty-print.  Let's ask for the solution by itself so we can see.

In [10]:
ms[0]

Metavariable,Instantiation
X,b
Y,a


Okay, the solution requires the instantiation $X=b$ and $Y=a$, but what is the proof?

In [11]:
ms[0].proof

"a, b, { :b :a and(b,a) } $~\vdash~$ and(b,a)",GL
"a, b, and(b,a) $~\vdash~$ and(b,a)",S
"a, b $~\vdash~$ { b a }",CR
"a, b $~\vdash~$ b",S
"a, b $~\vdash~$ { a }",CR
"a, b $~\vdash~$ a",S
"a, b $~\vdash~$ { }",T
,


Ooh, proofs pretty-print, too!  Nice.